<a name='0'></a>
# Overview

You use autocorrect every day on your cell phone and computer. In this notebook, you will explore what really goes on behind the scenes. Of course, the model you are about to implement is not identical to the one used in your phone, but it is still quite good. 

You will learn how to: 

- Get a word count given a corpus
- Get a word probability in the corpus 
- Manipulate strings 
- Filter strings 
- Implement Minimum edit distance to compare strings and to help find the optimal path for the edits. 
- Understand how dynamic programming works


Similar systems are used everywhere. 
- For example, if you type in the word **"I am lerningg"**, chances are very high that you meant to write **"learning"**, as shown in **Figure 1**. 
![fishy](image1.png)

<a name='0-1'></a>
# Edit Distance

You will implement models that correct words that are 1 and 2 edit distances away. 
- We say two words are n edit distance away from each other when we need n edits to change one word into another. 

An edit could consist of one of the following options: 

- Delete (remove a letter): ‘hat’ => ‘at, ha, ht’
- Switch (swap 2 adjacent letters): ‘eta’ => ‘eat, tea,...’
- Replace (change 1 letter to another): ‘jat’ => ‘hat, rat, cat, mat, ...’
- Insert (add a letter): ‘te’ => ‘the, ten, ate, ...’

You will be using the four methods above to implement an Auto-correct. 
- To do so, you will need to compute probabilities that a certain word is correct given an input. 

This auto-correct you are about to implement was first created by [Peter Norvig](https://en.wikipedia.org/wiki/Peter_Norvig) in 2007. 
- His [original article](https://norvig.com/spell-correct.html) may be a useful reference for your studying.

The goal of our spell check model is to compute the following probability:

$$P(c|w) = \frac{P(w|c)\times P(c)}{P(w)} \tag{1}$$

The equation above is [Bayes Rule](https://en.wikipedia.org/wiki/Bayes%27_theorem). 
- Equation 1 says that the probability of a word being correct $P(c|w) $is equal to the probability of having a certain word $w$, given that it is correct $P(w|c)$, multiplied by the probability of being correct in general $P(C)$ divided by the probability of that word $w$ appearing $P(w)$ in general.
- To compute equation 1, you will first import a data set and then create all the probabilities that you need using that data set. 

# **Part 1: Data Processing**

In [1]:
import re
from collections import Counter
import numpy as np
import pandas as pd
import os

In [2]:
def process_data(file_name):
  """
  Input:
    A file which you want to process
  Output: 
    words: a list containing all the words in the corpus (text file) with lower case
  """

  words = []
  with open(file_name) as f:
    data = f.read()
  words = re.findall(r'\w+', data.lower())
  return words

In [3]:
# Take data from path
path = os.getcwd()
file_name = path + '/big.txt'

words = process_data(file_name)
vocab = set(words)
print(f"The first ten words in the text are: \n{words[0:10]}")
print(f"There are {len(vocab)} unique words in the vocabulary.")

The first ten words in the text are: 
['the', 'project', 'gutenberg', 'ebook', 'of', 'the', 'adventures', 'of', 'sherlock', 'holmes']
There are 27041 unique words in the vocabulary.


In [4]:
word_count_dict = Counter(words)
print(f"There are {len(word_count_dict)} key values pairs")
print(f"10 most words in corpus are {word_count_dict.most_common(10)}")
print(f"The count for the word 'thee' is {word_count_dict.get('the',0)}")

There are 27041 key values pairs
10 most words in corpus are [('the', 56324), ('of', 29389), ('and', 23241), ('to', 17407), ('in', 15916), ('a', 14231), ('is', 7560), ('it', 6960), ('that', 6674), ('was', 6417)]
The count for the word 'thee' is 56324


Another way to get a list all words in the corpus and a dict which counting number of appearance of a word


```
def words(text): return re.findall(r'\w+', text.lower())
word_count_dict = Counter(words(open(file_name).read()))
```



Given the dictionary of word counts, compute the probability that each word will appear if randomly selected from the corpus of words.

$$P(w_i) = \frac{C(w_i)}{M} \tag{2}$$
where 

$C(w_i)$ is the total number of times $w_i$ appears in the corpus.

$M$ is the total number of words in the corpus.

For example, the probability of the word 'am' in the sentence **'I am happy because I am learning'** is:

$$P(am) = \frac{C(w_i)}{M} = \frac {2}{7} \tag{3}.$$

because the number of "am" is 2 and total words of sentence is 7.

In [5]:
def get_probs(word_count_dict):
    '''
    Input:
        word_count_dict: The wordcount dictionary where key is the word and value is its frequency.
    Output:
        probs: A dictionary where keys are the words and the values are the probability that a word will occur. 
    '''
    probs = {}  # return this variable correctly
    
    M = sum(word_count_dict.values())
    for key in word_count_dict.keys():
        probs[key] = word_count_dict[key] / M
        
    return probs

In [6]:
probs = get_probs(word_count_dict)
print(f"Length of probs is {len(probs)}")
print(f"P('the') is {probs['the']:.4f}")

Length of probs is 27041
P('the') is 0.0787


Another way to get probability of a word

```
def Prob(word, M=sum(word_count_dict.values())): 
    "Probability of `word`."
    return word_count_dict[word] / M
```



<a name='2'></a>
# **Part 2: String Manipulations**

Now, that you have computed $P(w_i)$ for all the words in the corpus, you will write a few functions to manipulate strings so that you can edit the erroneous strings and return the right spellings of the words. In this section, you will implement four functions: 

* `delete_letter`: given a word, it returns all the possible strings that have **one character removed**. 
* `switch_letter`: given a word, it returns all the possible strings that have **two adjacent letters switched**.
* `replace_letter`: given a word, it returns all the possible strings that have **one character replaced by another different letter**.
* `insert_letter`: given a word, it returns all the possible strings that have an **additional character inserted**. 

In [30]:
def edit_one_letter(word):
    "All edits that are one edit away from `word`."

    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    
    # L is left, R is right of the set in 'splits' list
    # For example, splits = [('', 'anh'), ('a', 'nh'), ('an', 'h'), ('anh', '')] 
    # ('a', 'nh') is a set in above list and L is 'a', R is 'nh'
    delete = [L + R[1:] for L, R in splits if R]
    switch = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replace = [L + c + R[1:] for L, R in splits if R for c in letters]
    insert = [L + c + R for L, R in splits for c in letters]

    return set(delete + switch + replace + insert)

def edit_two_letters(word):
    "All edits that are two edits away from `word`."
    return set(e2 for e1 in edit_one_letter(word) for e2 in edit_one_letter(e1))

In [31]:
tmp_edit_two_set = edit_two_letters("a")
tmp_edit_two_l = sorted(list(tmp_edit_two_set))

print(f"Number of strings with edit distance of two: {len(tmp_edit_two_l)}")
print(f"First 10 strings {tmp_edit_two_l[:10]}")
print(f"Last 10 strings {tmp_edit_two_l[-10:]}")
print(f"The data type of the returned object should be a set {type(tmp_edit_two_set)}")
print(f"Number of strings that are 2 edit distances from 'at' is {len(edit_two_letters('at'))}")

Number of strings with edit distance of two: 2654
First 10 strings ['', 'a', 'aa', 'aaa', 'aab', 'aac', 'aad', 'aae', 'aaf', 'aag']
Last 10 strings ['zv', 'zva', 'zw', 'zwa', 'zx', 'zxa', 'zy', 'zya', 'zz', 'zza']
The data type of the returned object should be a set <class 'set'>
Number of strings that are 2 edit distances from 'at' is 7154


# **Part 3: Suggest spelling suggestions**

In [9]:
def Prob(word, M=sum(word_count_dict.values())): 
    "Probability of `word`."
    return word_count_dict[word] / M

print(f"P('the') is {Prob('the'):.4f}")

P('the') is 0.0787


In [10]:
def known(words):
  "The subset of 'words' that appear in the dictionary of word_count_dict"
  return set(w for w in words if w in word_count_dict)

print(f"'somthing' can be {known(edit_two_letters('somthing'))}")

'somthing' can be {'smoothing', 'something', 'soothing', 'scathing', 'nothing', 'loathing'}


In [11]:
def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edit_one_letter(word)) or known(edit_two_letters(word)) or [word])
print(candidates('somthing'))

{'something', 'soothing'}


In [20]:
def correction(word, verbose=True): 
    "Most probable spelling correction for word."
    if verbose: 
        print(word, ": ", candidates(word))
    return max(candidates(word), key=Prob)

print(f"'dys' is changed '{correction('dys')}'\n")
print(f"'kerrectud' is changed '{correction('korrectud')}'\n")
print(f"'somthing' is changed '{correction('somthing')}'\n")

dys :  {'dye', 'dy', 'days', 'des'}
'dys' is changed 'days'

korrectud :  {'corrected'}
'kerrectud' is changed 'corrected'

somthing :  {'something', 'soothing'}
'somthing' is changed 'something'



# **Part 4: Minimum Edit Distance**
Now that you could implemented auto-correct, but how do you evaluate the similarity between two strings? For example: 'waht' and 'what'

Also how do you efficiently find the shortest path to go from the word, 'waht' to the word 'what'?

## Dynamic Programing
What is *dynamic programing*? 

Dynamic Programming breaks a problem down into subproblems which can be combined to form the final solution. Here, given a string source[0..i] and a string target[0..j], we will compute all the combinations of substrings[i, j] and calculate their edit distance. To do this efficiently, we will use a table to maintain the previously computed substrings and use those to calculate larger substrings.

You have to create a matrix and update each element in the matrix as follows: 

$$\text{Initialization}$$

\begin{align}
D[0,0] &= 0 \\
D[i,0] &= D[i-1,0] + del\_cost(source[i]) \tag{4}\\
D[0,j] &= D[0,j-1] + ins\_cost(target[j]) \\
\end{align}


$$\text{Per Cell Operations}$$
\begin{align}
 \\
D[i,j] =min
\begin{cases}
D[i-1,j] + del\_cost\\
D[i,j-1] + ins\_cost\\
D[i-1,j-1] + \left\{\begin{matrix}
rep\_cost; & if src[i]\neq tar[j]\\
0 ; & if src[i]=tar[j]
\end{matrix}\right.
\end{cases}
\tag{5}
\end{align}

So converting the source word **play** to the target word **stay**, using an input cost of one, a delete cost of 1, and replace cost of 2 would give you the following table:
<table style="width:20%">

  <tr>
    <td> <b> </b>  </td>
    <td> <b># </b>  </td>
    <td> <b>s </b>  </td>
    <td> <b>t </b> </td> 
    <td> <b>a </b> </td> 
    <td> <b>y </b> </td> 
  </tr>
   <tr>
    <td> <b>  #  </b></td>
    <td> 0</td> 
    <td> 1</td> 
    <td> 2</td> 
    <td> 3</td> 
    <td> 4</td> 
 
  </tr>
  <tr>
    <td> <b>  p  </b></td>
    <td> 1</td> 
 <td> 2</td> 
    <td> 3</td> 
    <td> 4</td> 
   <td> 5</td>
  </tr>
   
  <tr>
    <td> <b> l </b></td>
    <td>2</td> 
    <td>3</td> 
    <td>4</td> 
    <td>5</td> 
    <td>6</td>
  </tr>

  <tr>
    <td> <b> a </b></td>
    <td>3</td> 
     <td>4</td> 
     <td>5</td> 
     <td>4</td>
     <td>5</td> 
  </tr>
  
   <tr>
    <td> <b> y </b></td>
    <td>4</td> 
      <td>5</td> 
     <td>6</td> 
     <td>5</td>
     <td>4</td> 
  </tr>
  

</table>



The operations used in this algorithm are 'insert', 'delete', and 'replace'. These correspond to the functions that you defined earlier: insert_letter(), delete_letter() and replace_letter(). switch_letter() is not used here.

The diagram below describes how to initialize the table. Each entry in D[i,j] represents the minimum cost of converting string source[0:i] to string target[0:j]. The first column is initialized to represent the cumulative cost of deleting the source characters to convert string "EER" to "". The first row is initialized to represent the cumulative cost of inserting the target characters to convert from "" to "NEAR".

<div style="width:image width px; font-size:100%; text-align:center;"><img src='EditDistInit4.PNG' alt="alternate text" width="width" height="height" style="width:1000px;height:400px;"/> </div>  

Note that the formula for $D[i,j]$ shown in the image is equivalent to:

\begin{align}
 \\
D[i,j] =min
\begin{cases}
D[i-1,j] + del\_cost\\
D[i,j-1] + ins\_cost\\
D[i-1,j-1] + \left\{\begin{matrix}
rep\_cost; & if src[i]\neq tar[j]\\
0 ; & if src[i]=tar[j]
\end{matrix}\right.
\end{cases}
\tag{5}
\end{align}

The variable `sub_cost` (for substitution cost) is the same as `rep_cost`; replacement cost.  We will stick with the term "replace" whenever possible.

Below are some examples of cells where replacement is used. This also shows the minimum path from the lower right final position where "EER" has been replaced by "NEAR" back to the start. This provides a starting point for the optional 'backtrace' algorithm below.


<div style="width:image width px; font-size:100%; text-align:center;"><img src='EditDistExample1.PNG' alt="alternate text" width="width" height="height" style="width:1200px;height:400px;"/> Examples Distance Matrix</div>

In [17]:
def min_edit_distance(source, target, ins_cost = 1, del_cost = 1, rep_cost = 2):
    '''
    Input: 
        source: a string corresponding to the string you are starting with
        target: a string corresponding to the string you want to end with
        ins_cost: an integer setting the insert cost
        del_cost: an integer setting the delete cost
        rep_cost: an integer setting the replace cost
    Output:
        D: a matrix of len(source)+1 by len(target)+1 containing minimum edit distances
        med: the minimum edit distance (med) required to convert the source string to the target
    '''
    m = len(source)
    n = len(target)
    
    # Initialize a matrix with zeros and dimensions (m+1, n+1)
    D = np.zeros((m+1, n+1), dtype=int)
    
    # Initialize values of 0-th row and 0-th column (i=0, j=0)
    for row in range(1, m+1):
        D[row, 0] = D[row-1, 0] + del_cost
        
    for column in range(1, n+1):
        D[0, column] = D[0, column-1] + ins_cost
    
    # Calculate D[row-th, column-th]
    for row in range(1, m+1):
        for column in range(1, n+1):
            
            # Intialize r_cost to the 'replace' cost that is passed into this function
            r_cost = rep_cost
            
            # Update r_cost = 0 if element of source and target are the same
            if source[row-1] == target[column-1]:
                r_cost = 0
                
            # Update the cost at row, col based on previous entries in the cost matrix
            # Refer to the equation calculate for D[i,j] (the minimum of three calculated costs)
            D[row, column] = min(D[row-1, column] + del_cost, D[row, column-1] + ins_cost, D[row-1, column-1] + r_cost)
    
    med = D[m, n]
    
    return D, med

In [18]:
# Example 1 
source =  'play'
target = 'stay'
matrix, min_edits = min_edit_distance(source, target)
print("minimum edits: ",min_edits, "\n")
idx = list('#' + source)
cols = list('#' + target)
df = pd.DataFrame(matrix, index=idx, columns= cols)
print(df)

minimum edits:  4 

   #  s  t  a  y
#  0  1  2  3  4
p  1  2  3  4  5
l  2  3  4  5  6
a  3  4  5  4  5
y  4  5  6  5  4


In [19]:
# Example 2
source =  'eer'
target = 'near'
matrix, min_edits = min_edit_distance(source, target)
print("minimum edits: ",min_edits, "\n")
idx = list(source)
idx.insert(0, '#')
cols = list(target)
cols.insert(0, '#')
df = pd.DataFrame(matrix, index=idx, columns= cols)
print(df)

minimum edits:  3 

   #  n  e  a  r
#  0  1  2  3  4
e  1  2  1  2  3
e  2  3  2  3  4
r  3  4  3  4  3


In [39]:
source = "eer"
targets = edit_one_letter(source) 
for word in targets:
    _, min_edits = min_edit_distance(source, word, 1, 1, 1)  # set ins, del, sub costs all to one
    if min_edits != 1: print(source, word, min_edits)
#     print(source, word, min_edits)

eer ere 2
eer eer 0
